<a href="https://colab.research.google.com/github/davigm/An-lise-usando-SQL/blob/main/Projeto_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Enquanto estudava SQL, procurei uma forma de praticar e fixar o que aprendi. As opções tradicionais sugeridas (DB Browser, por exemplo) não permite a execução do código de forma interativa tal qual fazemos no Jupyter Notebook.

Na minha busca por melhores opções, encontrei este artigo https://towardsdatascience.com/have-a-sql-interview-coming-up-ace-it-using-google-colab-6d3c0ffb29dc que ensina como utilizar código SQL usando o Google Colab e consequentemente qualquer ambiente Python.

A tabela utilizada pode ser encontrada em https://data.transportation.gov/Aviation/Consumer-Airfare-Report-Table-1-Top-1-000-Contiguo/4f3n-jbg2 e foi sugerida em um forum do site Codecademy (https://discuss.codecademy.com/t/data-science-independent-project-5-analyze-airfare-data/419949) como forma de aplicar o conteúdo aprendido no curso de SQL.

In [1]:
import pandas as pd
import sqlite3

## SQL Hepler Functions

In [2]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

## Ler arquivo CSV

In [3]:
# Step 1: Read the csv file into a dataframe
# Dataset from https://www.kaggle.com/gpreda/covid-world-vaccination-progress
input_df = pd.read_csv('https://data.transportation.gov/api/views/4f3n-jbg2/rows.csv?accessType=DOWNLOAD')
 


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
input_df.shape

(102030, 18)

## Projeto: Analisar dados de tarifas aéreas
A tabela que será utilizada nesta análise possui registros das 1000 maiores rotas aéreas dos Estados Unidos. A tabela lista o número de passageiros com viagens apenas de ida por dia, a distância sem escalas percorrida, entre outas informações.

### Colunas do Dataset
* Year - Ano do vôo
* quarter - trimestre
* citymarketid_1 - ID da cidade 1
* citymarketid_2 - ID da cidade 2
* city1 - cidade 1
* city2 - cidade 2
* nsmiles - distância entre as cidades (em radiano)
* passengers - passageiros por dia
* fare - tarifa
* carrier_lg - Operadora com a maior participação de mercado
* large_ms - Participação da operadora com a maior participação de mercado
* fare_lg - Tarifa média para a transportadora com a maior participação de mercado
* carrier_low - Transportadora com a tarifa mais baixa
* lf_ms - Participação de mercado para a transportadora com a tarifa mais baixa
* fare_low - Tarifa média para a transportadora com a tarifa mais baixa 
* table_1_flag - flag
* Geocoded_City1 - coordenadas geográficas da cidade 1
* Geocoded_City2 - coordenadas geográficas da cidade 2

## Fazer upload do dataframe para a tabela SQL

In [5]:
# Upload do dataframe para uma tabela SQL
pd_to_sqlDB(input_df,
            table_name='consumer_report',
            db_name='default.db')


2021-11-23 10:40:10 INFO: SQL DB default.db created
2021-11-23 10:40:10 INFO: SQL Table consumer_report created with 18 columns
2021-11-23 10:40:12 INFO: 102030 rows uploaded to consumer_report


## Análise
### Qual o *range* de anos?

In [6]:
# Determinando o menor valor da coluna Year
sql_query_string = """
    SELECT MIN(Year)
    FROM consumer_report
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,MIN(Year)
0,1996


In [7]:
# Determinando o maior valor da coluna Year
sql_query_string = """
    SELECT MAX(Year)
    FROM consumer_report
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,MAX(Year)
0,2021


**O range da coluna Year é 2021-1996 = 25 anos**

### Quais são as distâncias de vôos mais curta e mais longas entre 2 cidades?

In [8]:
# Determinando a menor distâcia entre duas cidades.
sql_query_string = """
    SELECT city1,city2,MIN(nsmiles)
    FROM consumer_report
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,city1,city2,MIN(nsmiles)
0,"Los Angeles, CA (Metropolitan Area)","San Diego, CA",109


In [9]:
# Determinando a maior distâcia entre duas cidades.
sql_query_string = """
    SELECT city1,city2,MAX(nsmiles)
    FROM consumer_report
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,city1,city2,MAX(nsmiles)
0,"Miami, FL (Metropolitan Area)","Seattle, WA",2724


### Quantas cidades distintas são representadas?




In [10]:
# Determinando o número de cidades distintas.
sql_query_string = """
    SELECT city1
    FROM consumer_report
    UNION
    SELECT city2
    FROM consumer_report
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,city1
0,"Albany, NY"
1,"Albuquerque, NM"
2,"Allentown/Bethlehem/Easton, PA"
3,"Amarillo, TX"
4,"Appleton, WI"
...,...
170,"Washington, DC (Metropolitan Area)"
171,"West Palm Beach/Palm Beach, FL"
172,"Wichita, KS"
173,"Wilmington, DE"


**Existem 175 cidades distintas**

###Qual companhia aérea aparece com mais frequência como a transportadora com a tarifa mais baixa

In [11]:
sql_query_string = """
    SELECT carrier_low, COUNT(carrier_low)
    FROM consumer_report
    GROUP BY carrier_low
    ORDER BY COUNT(carrier_low) DESC
    LIMIT 5;
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,carrier_low,COUNT(carrier_low)
0,WN,35144
1,DL,8803
2,AA,8283
3,US,6527
4,FL,5997


**A companhia que apresenta a menor tarifa com mais frequência é a WN, aparecendo 35144 vezes como a menor tarifa**

###Qual companhia aérea aparece com mais frequência como a companhia com a maior fatia de mercado.

In [12]:
sql_query_string = """
    SELECT carrier_lg, COUNT(carrier_lg)
    FROM consumer_report
    GROUP BY carrier_lg
    ORDER BY COUNT(carrier_lg) DESC
    LIMIT 5;
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,carrier_lg,COUNT(carrier_lg)
0,WN,27381
1,DL,18128
2,AA,14163
3,UA,9259
4,US,8949


**A companhia que apresenta a maior fatia de mercado com mais frequência é a WN, aparecendo 27381 vezes na tabela**

###Quantas ocorrências existem em que a transportadora com a maior participação de mercado não é a transportadora com a tarifa mais baixa? 



In [22]:
sql_query_string = """
    SELECT COUNT(carrier_lg)
    FROM consumer_report
    WHERE carrier_low <> 'WN';
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,COUNT(carrier_lg)
0,66883


**Existem 66883 ocorrências em que a transportadora com a maior participação de mercado não é a transportadora com a tarifa mais baixa**

In [24]:
sql_query_string = """
    SELECT ROUND(AVG(fare_lg - fare_low), 2) AS 'Average Difference in Fare'
    FROM consumer_report
    WHERE carrier_low != carrier_lg;
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,Average Difference in Fare
0,51.06


###Qual a mudança pecentual por vôo da média da taxa nos periodos: 2007 a 2017 e entre 1997 e 2017?

In [27]:
sql_query_string = """
    SELECT fare_lg - fare_low AS 'Diferença da taxa(2007 a 2017)'
    FROM consumer_report
    WHERE Year BETWEEN 2007 AND 2017
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,Diferença da taxa(2007 a 2017)
0,0.00
1,32.60
2,43.53
3,97.36
4,92.90
...,...
44003,0.00
44004,0.00
44005,40.70
44006,62.66


In [29]:
sql_query_string = """
    SELECT fare_lg - fare_low AS 'Diferença da taxa(1997 a 2017)'
    FROM consumer_report
    WHERE Year BETWEEN 1997 AND 2017
"""
 
# Executa a consulta SQL
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,Diferença da taxa(1997 a 2017)
0,0.00
1,67.83
2,32.60
3,68.03
4,43.53
...,...
84012,0.00
84013,0.00
84014,40.70
84015,62.66
